In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML
from pathlib import Path

from classes import TractPlot
from demographics import DemPlot

In [2]:
from bqplot import Figure, Scatter, Axis, LinearScale

In [3]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [4]:
# app = App.from_csv("indicators.csv")
sub = '996782'

# pull data from s3
# fname = f's3://open-neurodata/rokem/hcp1200/afq/sub-{sub}/ses-01/sub-{sub}_dwi_space-RASMM_model-CSD_desc-prob-afq_profiles.csv'


# pull data from local data directory
data_path = Path('data')
subdir = f'sub-{sub}/ses-01/'
fname = f'sub-{sub}_dwi_space-RASMM_model-CSD_desc-prob-afq_profiles.csv'
filepath = data_path / subdir / fname
print(filepath)
tract_interact = TractPlot.from_csv(filepath)

behav_local = '../../hcp_behaviorals.csv'
demograph = DemPlot.from_csv(behav_local)

# app.container

data/sub-996782/ses-01/sub-996782_dwi_space-RASMM_model-CSD_desc-prob-afq_profiles.csv
   Unnamed: 0 tractID  nodeID    dki_fa    dki_md    dki_mk   dki_awf
0           0   ATR_R       0  0.244326  0.000919  0.567310  0.214483
1           1   ATR_R       1  0.299771  0.000935  0.666079  0.243779
2           2   ATR_R       2  0.334685  0.000961  0.745118  0.268511
3           3   ATR_R       3  0.370716  0.000972  0.793672  0.287652
4           4   ATR_R       4  0.402095  0.000966  0.829839  0.303777
   Subject Release Acquisition Gender    Age  3T_Full_MR_Compl  T1_Count  \
0   100004    S900         Q06      M  22-25             False         0   
1   100206    S900         Q11      M  26-30              True         1   
2   100307      Q1         Q01      F  26-30              True         1   
3   100408      Q3         Q03      M  31-35              True         1   
4   100610    S900         Q08      M  26-30              True         2   

   T2_Count  3T_RS-fMRI_Count  3T_RS

Output()

In [5]:
# just temporarily testing things:
# df = pd.read_csv(behav_local)
# age = df['T1_Count']
# import bqplot.pyplot as plt
# plt.hist(age)

In [6]:
# Items that populate each section of the dashboard get defined here

# List of items in the Demographics section:
demogr_items = [
    widgets.HTML((
        '<h4>Demographics Information</h4>'), 
                 layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    demograph.container
    # add more widgets here
]

# List of items in the Behavioural section:
behav_items = [
    widgets.HTML(('<h4>Behavioural Data</h4>'), layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    # add more widgets here, for example the demo plotter class
]

# List of items in the Neuroimaging section:
neuro_items = [
    widgets.HTML((
        '<h4>Neuroimaging Data</h4>'
        '<p>Select which tract and which DKI measure to display:<p>'), 
                 layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    tract_interact.container # add more widgets here
]

In [7]:
# Render the different sections of the app

# The header section contains only the main title and subtitle of the website as an HTML widget:
header           = widgets.HTML((
                        '<h1>Explore the Human Connectome Project Young Adult dataset!</h1>'
                        '<p>Select what data to plot in the dropdowns of each section</p>'), 
                        layout=widgets.Layout(width='auto', grid_area='header'))

# The demographics section is a "box" (container) that contains all of the widgets listed under the demogr_items (defined in cell above):
demographics     = widgets.VBox(children=demogr_items,
                        layout=widgets.Layout(width='auto', grid_area='demographics', border='2px solid grey', align_items='center'))

# The behavioural section is a "box" (container) that contains all of the widgets listed under the behav_items (defined in cell above):
behavioural      = widgets.VBox(children=behav_items,
                        layout=widgets.Layout(width='auto', grid_area='behavioural', border='2px solid grey', align_items='center'))

# The neuroimaging section is a "box" (container) that contains all of the widgets listed under the neuro_items (defined in cell above):
neuroimaging     = widgets.VBox(children=neuro_items,
                        layout=widgets.Layout(width='auto', grid_area='neuroimaging', border='2px solid grey', align_items='center'))

# The footer section contains only the footer string and related hyperlinks as an HTML widget:
footer           = widgets.HTML((
                        '<p><i>Created at NeuroHackademy 2024 using the <b style="color:blue"><a href="https://github.com/voila-dashboards/voici-demo"> Voici demo template repository</a></b>. | <b style="color:blue"><a href="https://github.com/NeuroHackademy2024/neuro-nav">Link to code</a></b></i><p>'),
                        layout=widgets.Layout(width='auto', grid_area='footer'))

widgets.GridBox(children=[header, demographics, behavioural, neuroimaging, footer],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "header header header header"
            "demographics demographics behavioural behavioural"
            "neuroimaging neuroimaging neuroimaging neuroimaging"
            "footer footer footer footer"
            ''')
       )

GridBox(children=(HTML(value='<h1>Explore the Human Connectome Project Young Adult dataset!</h1><p>Select what…